# Product LookUp Langchain

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents import AgentExecutor

In [ ]:
api_key = "sk-xxxxxxx"

In [3]:
fake_db = {
        "macbook": {
            "name": "MacBook Air M2",
            "price": "$999"
        },
        "ipad": {
            "name": "iPad Pro 12.9",
            "price": "$1099"
        },
        "iphone": {
            "name": "iPhone 15 Pro",
            "price": "$1199"
        }
    }

In [4]:
# 🧰 Define a simple product lookup tool
def product_lookup(product_name: str) -> str:
    product = fake_db.get(product_name.lower())
    if product:
        return f"{product['name']} - {product['price']}"
    return "Product not found."


In [5]:
# 🧰 Wrapping the function as a tool
tools = [
    Tool(
        name="Product Lookup",
        func=product_lookup,
        description="Use this tool to get product details by name."
    )
]

In [6]:
# ⚙️ Set up the GPT-3.5 Model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=api_key)

/tmp/ipykernel_9512/1661257993.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=api_key)


In [9]:
# 🧑‍💼 Create and initialize the agent
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True,
    agent_kwargs={
        "system_message": "Only share the product price with me"
    }
)

In [10]:
print("Agent is ready. Ask about a product (e.g., macbook, ipad, iphone).")
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    response = agent.run(user_input)
    print("Agent:", response)


Agent is ready. Ask about a product (e.g., macbook, ipad, iphone).


> Entering new AgentExecutor chain...
I should use the Product Lookup tool to get details about the iPad.
Action: Product Lookup
Action Input: "iPad"
Observation: iPad Pro 12.9 - $1099
Thought:I need to know the price of the iPad Pro 12.9
Action: Product Lookup
Action Input: "iPad Pro 12.9"
Observation: Product not found.
Thought:I should try searching for a different version of the iPad Pro 12.9.
Action: Product Lookup
Action Input: "iPad Pro 12.9 inch"
Observation: Product not found.
Thought:I should try searching for a more general term for the iPad Pro 12.9.
Action: Product Lookup
Action Input: "iPad Pro"
Observation: Product not found.
Thought:It seems like the specific iPad Pro 12.9 version is not available in the product database. I should consider looking for a different iPad model.
Action: Product Lookup
Action Input: "iPad"
Observation: iPad Pro 12.9 - $1099
Thought:I now know the final answer
Final Answer: 

# Product LookUp LangGraph

In [11]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableLambda

In [12]:
fake_db = {
    "macbook": {
        "name": "MacBook Air M2",
        "price": "$999"
    },
    "ipad": {
        "name": "iPad Pro 12.9",
        "price": "$1099"
    },
    "iphone": {
        "name": "iPhone 15 Pro",
        "price": "$1199"
    }
}


In [17]:
# ---🔍 Product lookup logic---
def lookup_product(state: dict) -> dict:
    print("States in LookUp Product : ", state)
    user_input = state["input"]
    print("User Input is : ", user_input)
    product = fake_db.get(user_input.lower())
    print("product Input is : ", user_input)
    if product:
        return {
            "product_info": f"{product['name']} - {product['price']}"
        }
    return {
        "product_info": "Product not found."
    }

In [ ]:
api_key = "sk-key"

In [18]:
# ⚙️ Set up the GPT-3.5 Model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=api_key)

In [19]:
def respond_with_llm(state: dict) -> dict:
    print("States in Respond with LLM : ", state)
    product_info = state["product_info"]
    print("Product Input is : ", product_info)
    prompt = f"You are a helpful assistant. Here is the product information: {product_info}"
    response = llm.invoke(prompt)
    return {"output": response.content}

In [21]:
from typing import TypedDict, Optional

class ProductGraphState(TypedDict):
    input: str
    product_info: Optional[str]
    output: Optional[str]


**🧩 Nodes = Steps**

A node is a specific unit of work — it does something.

In LangGraph, each node is a function (like product lookup, LLM response, etc.).

Nodes take a state (dict) as input and return a new state.

***Examples:***

lookup_product: Looks up the product in the DB.

respond: Uses LLM to respond with product info.

clarify: Asks the user for clarification.


**🔗 Edges = Connections**

An edge defines the path or transition from one node to the next.

Think of edges like "after this step, go to that step."

You can add conditions to edges to create branches or loops.

***Examples:***

From lookup_product → respond

From respond → END

From lookup_product → clarify (if product not found)

In [22]:
# ---🔧 Create the graph---
graph = StateGraph(ProductGraphState)

# Nodes
graph.add_node("lookup_product", RunnableLambda(lookup_product))
graph.add_node("respond", RunnableLambda(respond_with_llm))

# Edges
graph.set_entry_point("lookup_product")
graph.add_edge("lookup_product", "respond")
graph.add_edge("respond", END)

# ---🚀 Compile the graph---
app = graph.compile()


In [25]:
print("🔍 Ask about a product (macbook, iphone, or ipad). Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    result = app.invoke({"input": user_input})
    print("Assistant:", result["output"])


🔍 Ask about a product (macbook, iphone, or ipad). Type 'exit' to quit.
States in LookUp Product :  {'input': 'iphone'}
User Input is :  iphone
product Input is :  iphone
States in Respond with LLM :  {'input': 'iphone', 'product_info': 'iPhone 15 Pro - $1199'}
Product Input is :  iPhone 15 Pro - $1199
Assistant: Thank you for providing the product information. The iPhone 15 Pro is priced at $1199. If you have any further questions or need assistance, feel free to ask.
